# Sub Question Query Engine
In this tutorial, we showcase how to use a **sub question query engine** to tackle the problem of answering a complex query using multiple data sources.  
It first breaks down the complex query into sub questions for each relevant data source,
then gather all the intermediate reponses and synthesizes a final response.

### Preparation

In [9]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.query_engine import SubQuestionQueryEngine

In [10]:
# load data
pg_essay = SimpleDirectoryReader(input_dir="../data/paul_graham/").load_data()

# build index and query engine
query_engine = GPTVectorStoreIndex.from_documents(pg_essay).as_query_engine()

### Setup sub question query engine

In [16]:
# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine, 
        metadata=ToolMetadata(name='pg_essay', description='Paul Graham essay on What I Worked On')
    )
]

query_engine = SubQuestionQueryEngine.from_defaults(query_engine_tools=query_engine_tools)

### Run queries

In [17]:
response = await query_engine.aquery('How was Paul Grahams life different before and after YC?')

Generated 2 sub questions.
[pg_essay] Q: What did Paul Graham work on before YC?
[pg_essay] Q: What did Paul Graham work on after YC?
Generated 1 sub questions.
[pg_essay] Q: What did Paul Graham work on after YC?
[pg_essay] A: 
After YC, Paul Graham continued to write essays and work on other projects. He wrote Hacker News in Arc, and worked on a new version of Arc with Robert Morris. He also worked on other projects, such as trying to learn the most he could about startups in the shortest possible time.
[pg_essay] A: 
After YC, Paul Graham worked on writing essays, creating Hacker News in Arc, working on a new version of Arc with Robert Morris, and learning as much as he could about startups.
Generated 1 sub questions.
[pg_essay] Q: What did Paul Graham work on before YC?
[pg_essay] A: 
Before YC, Paul Graham worked on hacking, writing essays, and a new version of Arc. He was also working on a news aggregator for startup founders called Startup News, which he later changed to Hacker 

In [18]:
print(response)


Before YC, Paul Graham worked mainly on hacking, writing essays, and creating a news aggregator for startup founders. After YC, Paul Graham shifted his focus to writing essays, creating Hacker News in Arc, working on a new version of Arc with Robert Morris, and learning as much as he could about startups. This shift in focus reflects a change in Paul Graham's life from mainly working on coding and writing to focusing more on startups and learning about the industry.
